Carlos Bravo Garrán - 100474964

 # __Exploratory Data Analysis (EDA)__ 

 In this notebook, a simplified Exploratory Data Analysis (EDA) will be performed on the provided dataset, with the objective of analyzing and understanding the factors that influence employee attrition in an organization.

### 1. __Load Libraries and Data__


In [ ]:
# Data analysis and wrangling
import numpy as np
import pandas as pd
import time

# Graphs
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Machine learning
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.metrics import (accuracy_score, balanced_accuracy_score, recall_score, roc_curve, auc, confusion_matrix, classification_report)
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier


# Warnings configuration
import warnings
warnings.filterwarnings('ignore')

Create virtual environment and install dependencies

In [ ]:
# Install venv and requirements
# !python3 -m venv venv
# !source venv/bin/activate
# %pip install -r ../requirements.txt

Load the dataset from the CSV file


In [ ]:
df = pd.read_csv("../data/attrition_availabledata_03.csv")

### 2. __Initial Exploration__

Review the general structure of the dataset


In [ ]:
print(df.info())

In [ ]:
dataset_shape = df.shape
print(f"The dataset contains {dataset_shape[0]} rows and {dataset_shape[1]} columns.")

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df[['Attrition']].head()

This is a __classification__ problem, as the target variable (Attrition) is binary (Yes / No). This means that the model must predict whether an employee will leave the company or not, rather than predicting a numerical value.

### 3. __Identify Categorical and Numerical Variables__

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
print("Categorical variables:", categorical_columns)
print("Numerical variables:", numerical_columns)

#### 3.1 Reclassify variables by adding ordinals

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
ordinal_columns = ["Education", "JobLevel", "EnvironmentSatisfaction", "JobSatisfaction", "WorkLifeBalance", "PerformanceRating", "StockOptionLevel"]
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()

# Remove from numerical the ones we have classified as ordinal
numerical_columns = [col for col in numerical_columns if col not in ordinal_columns]

print("Categorical variables:", categorical_columns)
print("Ordinal variables:", ordinal_columns)
print("Numerical variables:", numerical_columns)


#### 3.2 Detect high cardinality categorical variables

Identify categorical variables that may generate too many columns when encoding them


In [ ]:
categorical_cardinality = df[categorical_columns].nunique().sort_values(ascending=False)
display(categorical_cardinality)


It is not considered that there are high cardinality categorical variables, therefore no additional grouping or different encoding will be needed.

### 4. __Analysis of the Target Variable__

Review the distribution of the target variable to identify class imbalance

In [ ]:
if "Attrition" in df.columns:
    plt.figure(figsize=(4,4))
    sns.countplot(x=df["Attrition"], palette="viridis")
    plt.title("Distribution of the Target Variable (Attrition)\n")
    plt.show()
    
    attrition_counts = df["Attrition"].value_counts(normalize=True)
    display(pd.DataFrame(attrition_counts).rename(columns={"Attrition": "Proportion"}).reset_index(drop=True)*100)


In [ ]:
df.Attrition.value_counts().sort_index().to_frame()

The dataset is imbalanced, with 2466 employees not leaving the company (NO) and 474 who do leave (YES).

This means that the majority of employees do not leave the company, which could cause a poorly trained model to always predict "No", achieving an apparently high accuracy, but without actually capturing the cases of attrition.


### 5. __Identify Missing Values__

In [ ]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].to_frame().reset_index()
missing_values.columns = ["Column Name", "Missing Values"]

display(missing_values)

Since the missing values are very few compared to the total number of data, we will take the following measures:
- Impute missing values in numerical variables with the median
- Impute missing values in categorical variables with the mode

In [ ]:
# Columns differentiated by type
num_cols = [col for col in df.select_dtypes(include=['number']).columns if col not in ordinal_columns]
ord_cols = ordinal_columns
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

### 6. __Identify Constant and Identifying Variables__

#### 6.1 Check for columns with unique values

In [ ]:
unique_values = df.nunique()

constant_columns = df.nunique()[df.nunique() == 1].to_frame().reset_index()
constant_columns.columns = ["Column Name", "Unique Value Count"]
constant_columns["Unique Value"] = constant_columns["Column Name"].apply(lambda col: df[col].unique()[0])
constant_columns_list = constant_columns["Column Name"].tolist()

display(constant_columns)

#### 6.2 Check for columns with ID variables

In [ ]:
print(f"Total number of rows: {len(df)}")

unique_values = df.nunique()
print("Number of unique values per column:")
print(unique_values, "\n")

id_columns = [col for col in df.columns if df[col].nunique() == len(df)]
print("Identifying columns detected:", id_columns)

As its name indicates, _EmployeeID_ is an identifying variable, making it redundant for the study.

On the other hand, the _hours_ column has 2939 values, 1 less than the total number of rows. It might be considered identifying, but upon checking the values, we see that they are simply different decimal values.

#### 6.3 Remove Constant and Identifying Columns

In [ ]:
df = df.drop(columns=constant_columns_list + id_columns, errors='ignore')
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()

print('The following columns have been removed:', constant_columns_list + id_columns)


### 7. __Create Correlation Matrix__

Generate the correlation matrix to understand relationships between numerical variables, having already removed constant and identifying variables.

In [ ]:
df_numeric = df.select_dtypes(include=['number'])

correlation_matrix = df_numeric.corr()

plt.figure(figsize=(18, 18))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", linewidths=0.5, cmap="coolwarm")
plt.xticks(rotation=75)
plt.title("Correlation Matrix\n")
plt.show()


There are no extremely high correlations, close to 1 or -1, so the variables are not redundant nor strongly dependent on each other. However, there are some moderate correlations that we can consider.

__Relationship between tenure and work experience:__

- _YearsAtCompany_ and _YearsWithCurrManager_ (0.76): Employees who have been with the company longer are more likely to have been with the same manager for a longer time.
- _YearsAtCompany_ and _TotalWorkingYears_ (0.62): The longer a person has worked in general, the more time they may have spent at the current company.

__Relationship between _PercentSalaryHike_ and _PerformanceRating_ (0.78):__ There is a high correlation between salary increase and performance rating. Employees with better performance receive higher salary increases.

- It could be evaluated if _PercentSalaryHike_ is redundant, as it is strongly linked to _PerformanceRating_.


Regarding the rest, they have very low correlations with all the others, indicating that they may be independent or influenced by other factors not considered. Thus, it could be reviewed if these variables have any significant impact on the target variable, or if they can be eliminated.


### 8. __Identify Correlation with the Target Variable__

In [ ]:
# Convert the target variable to numeric
df_aux = df.copy()
df_aux["Attrition"] = df_aux["Attrition"].map({"Yes": 1, "No": 0})

# Ensure we only work with numeric columns
df_corr = df_aux.select_dtypes(include=['number'])
attrition_correlation = df_corr.corr()["Attrition"].sort_values()

plt.figure(figsize=(10, 6))
ax = sns.barplot(y=attrition_correlation.index, x=attrition_correlation.values, palette="coolwarm")

# Add values on the bars
for index, value in enumerate(attrition_correlation.values):
    ax.text(value, index, f"{value:.2f}", ha="left", va="center", fontsize=10, color="black")

plt.title("Correlation of 'Attrition' with other variables")
plt.xlabel("Correlation")
plt.ylabel("Variables")
plt.show()


Se puede comprobar que no hay una variable con una correlación extremadamente fuerte con Attrition, pero sí que muchas de ellas tienen una relación muy baja, por ello cabría la posibilidad de considerar su elimiación con el fin de simplificar el modelo.

Junto con las horas trabajadas, las variables de antigüedad (_YearsAtCompany_, _TotalWorkingYears_) son las que más influyen en la retención, se pueden evaluar las relaciones con _YearsWithCurrManager_ para comprobar si esta última sería redundante.

- Se podrían eliminar las variables que tienen muy baja correlación con la objetivo.

In [ ]:
# Filter variables with correlation less than 0.05 in absolute value
low_corr_columns = attrition_correlation[abs(attrition_correlation) < 0.05].index.tolist()

# Remove these columns from the dataset
df_filtered = df.drop(columns=low_corr_columns, errors='ignore')

print(f"The following columns have been removed in a test dataframe due to low correlation: {low_corr_columns}")


### __9. Visualize Relationships Between Correlated Variables__
Perform a visual exploration of the relationships between highly correlated variables to help decide if there are redundancies or if some variables need to be transformed before using them in a predictive model.

In [ ]:
# Visualize relationships between tenure variables with pairplot
sns.pairplot(df, vars=["YearsAtCompany", "YearsWithCurrManager", "TotalWorkingYears"], diag_kind="kde")
plt.suptitle("Relationships between Tenure and Work Experience", y=1.02)
plt.show()


__Relationship between Tenure and Work Experience__

- _YearsAtCompany_ and _YearsWithCurrManager_ (0.76): Strong relationship, possible redundancy.
- _YearsAtCompany_ and _TotalWorkingYears_ (0.62): Expected relationship, but provides different information.
- Conclusion: _YearsAtCompany_ or _YearsWithCurrManager_ could be redundant.

It could be evaluated for impact on the model and one could be removed if necessary.


Having identified and evaluated the variables, checking for null values, constant and identifying values, and correlations between them, we can proceed to the evaluation of potential classification models.

## __Evaluation of Classification Models with Advanced Preprocessing__
In this section, a preprocessing and modeling pipeline is implemented to predict the Attrition variable in an employee dataset. Cross-validation will be used for internal evaluation (inner evaluation) and a final evaluation with an independent test set (outer evaluation).

The main steps followed are:

1. Splitting data into training and test sets (2/3 for training the model - 1/3 for evaluating final performance).
2. Data preprocessing, including imputation, scaling, encoding, and dimensionality reduction.
3. Internal evaluation (inner evaluation) using stratified cross-validation.
4. Training and final evaluation (outer evaluation) with key metrics such as balanced accuracy, accuracy, TPR, TNR, and confusion matrix.

### __1. Data Splitting into Train and Test__

The predictor variables (X) are separated from the target variable (y) and the data is split into training and test sets.Se separan las variables predictoras (X) de la variable objetivo (y) y realiza la división de los datos en conjuntos de entrenamiento y prueba

In [ ]:
X = df.drop(columns=["Attrition"])
y = df["Attrition"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=1/3, random_state=100474964)

### __2. Data Preprocessing__
The preprocessing is divided into three types of variables:

- Numerical variables: Imputed with KNNImputer and scaled with RobustScaler.
- Categorical variables: Imputed with the mode (most_frequent) and encoded with OneHotEncoder, followed by dimensionality reduction with PCA(n_components=5).
- Ordinal variables: Imputed with the median and transformed with OrdinalEncoder.


#### 2.1 Identification of Variable Types


In [ ]:
numerical_columns = X.select_dtypes(include=['number']).columns.tolist()
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
ordinal_columns = ["Education", "JobLevel", "EnvironmentSatisfaction", "JobSatisfaction", "WorkLifeBalance", "PerformanceRating", "StockOptionLevel"]


#### 2.2 Comparison Between Means and Medians


In [ ]:
# Select columns with significant differences between mean and median
selected_columns = [
    "MonthlyIncome", "YearsAtCompany", "YearsSinceLastPromotion",
    "TotalWorkingYears", "DistanceFromHome", "NumCompaniesWorked"
]

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 12))
axes = axes.flatten()

for i, col in enumerate(selected_columns):
    sns.histplot(df[col], kde=True, bins=30, color='blue', ax=axes[i]) 
    axes[i].axvline(df[col].mean(), color='red', linestyle='--', label='Mean')
    axes[i].axvline(df[col].median(), color='green', linestyle='--', label='Median')
    axes[i].set_title(f"Distribution of {col}")
    axes[i].legend()

plt.tight_layout()
plt.show()

1. **Significant Differences Between Mean and Median**: Variables such as _MonthlyIncome_, _YearsAtCompany_, and _YearsSinceLastPromotion_ show notable differences between their means and medians, indicating skewed distributions or outliers.  
2. **Skewed Distributions**: Positive skew in variables like _MonthlyIncome_ and _YearsAtCompany_ suggests that means are influenced by extreme values, while medians are more robust.  
3. **Robustness of the Median**: In skewed distributions, the median is less sensitive to outliers, making it more suitable for describing the center of the data and imputing missing values.  

Given that many variables show significant differences between mean and median, along with skewed distributions, it is concluded that the __median__ is a more robust and representative strategy for imputing missing data.


#### 2.3 Transformations by Variable Type

In [ ]:
# Apply Label Encoding to ordinal variables
ord_encoder = OrdinalEncoder()
df[ordinal_columns] = ord_encoder.fit_transform(df[ordinal_columns])

ord_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

# Pipeline for categorical data
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ('pca', PCA(n_components=5))
])

# Pipeline for numerical data (imputation with median)
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

# ColumnTransformer with all preprocessing steps
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, numerical_columns),
    ('cat', cat_transformer, categorical_columns),
    ('ord', ord_transformer, ordinal_columns)
])

preprocessor


#### __2.4 Evaluation of Models with SMOTE__

In this section, tests are conducted by applying **SMOTE (Synthetic Minority Over-sampling Technique)** to evaluate its impact on the performance of classification models.

SMOTE allows balancing the class distribution, reducing the model's bias toward the majority class.


In [ ]:
from imblearn.over_sampling import SMOTE
# Apply the preprocessing pipeline to X_train
X_train_transformed = preprocessor.fit_transform(X_train)

# Apply SMOTE to the transformed data
smote = SMOTE(random_state=100474964)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_transformed, y_train)

print(f"Distribution after applying SMOTE:\n{pd.Series(y_train_resampled).value_counts()}")

Tras aplicar SMOTE, se ha podido ver que los resultados obtenidos muestran una ligera mejora en las métricas de rendimiento, especialmente en el True Positive Rate (TPR), lo que indica que los modelos son más efectivos en la detección de la clase minoritaria. 

Además, el coste computacional de aplicar SMOTE ha sido bajo, lo que lo convierte en una técnica viable para mejorar el equilibrio de clases sin afectar significativamente la eficiencia del entrenamiento y la inferencia.

### __3. Conversion of Target Variable (Attrition)__
Convert the Attrition variable to a binary format (1 for "Yes" and 0 for "No")

In [ ]:
y_train = y_train.map({"Yes": 1, "No": 0})
y_test = y_test.map({"Yes": 1, "No": 0})


### __4. Application of the Preprocessor__
The preprocessor is fitted and transforms the training and test data

In [ ]:
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [ ]:
def evaluate_model(model, X_test, y_test, model_name):
    """
    Evaluates a classification model and displays key metrics, confusion matrix, and ROC curve.
    """
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
    
    acc = accuracy_score(y_test, y_pred)
    bal_acc = balanced_accuracy_score(y_test, y_pred)
    tpr = recall_score(y_test, y_pred)  # Sensitivity / TPR
    tnr = recall_score(y_test, y_pred, pos_label=0)  # Specificity / TNR
    conf_matrix = confusion_matrix(y_test, y_pred)

    print(f"=== Final Evaluation: {model_name} ===")
    print(f"Accuracy: {acc:.4f}")
    print(f"Balanced Accuracy: {bal_acc:.4f}")
    print(f"TPR (Sensitivity): {tpr:.4f}")
    print(f"TNR (Specificity): {tnr:.4f}")
    print("\nConfusion Matrix:")
    print(classification_report(y_test, y_pred))

    plt.figure(figsize=(4, 4))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title(f"Confusion Matrix ({model_name})")
    plt.show()

    if y_prob is not None:
        fpr, tpr_curve, _ = roc_curve(y_test, y_prob)
        auc_score = auc(fpr, tpr_curve)

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr_curve, label=f'{model_name} (AUC = {auc_score:.4f})', color='blue')
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  
        
        plt.xlabel("False Positive Rate (FPR)")
        plt.ylabel("True Positive Rate (TPR)")
        plt.title(f"ROC Curve for {model_name}")
        plt.legend()
        plt.grid()
        plt.show()
    
        return {"Accuracy": acc, "Balanced Accuracy": bal_acc, "TPR": tpr, "TNR": tnr, "AUC": auc_score}
    
    return {"Accuracy": acc, "Balanced Accuracy": bal_acc, "TPR": tpr, "TNR": tnr, "AUC": None}

### __5. Evaluation with Dummy Model__

A Dummy model is trained to establish reference points:

- Expected Balanced Accuracy if the model were trivial.
- Comparison with trained models to demonstrate their effectiveness.

This will verify if the models are truly learning patterns or simply reflecting class distribution.

In [ ]:
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train_transformed, y_train)
dummy_ev = evaluate_model(dummy, X_test_transformed, y_test, "Dummy Model")


### __6. Definition of Models with Pipeline__
A pipeline is built that includes preprocessing and the classification model (_DecisionTreeClassifier_ and _KNeighborsClassifier_).

In [ ]:
clf_tree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', tree.DecisionTreeClassifier(class_weight='balanced'))
])

clf_tree

In [ ]:
clf_knn = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier(n_neighbors=5))
])

clf_knn

### __7. Internal Evaluation (Inner Evaluation)__
For internal evaluation, stratified cross-validation (StratifiedKFold) with 5 splits (n_splits=5) is used.

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=100474964)
cross_val_scores = cross_val_score(clf_tree, X_train, y_train, cv=skf, scoring='balanced_accuracy')
print(f"Balanced Accuracy (inner evaluation): {np.mean(cross_val_scores):.4f}")

### __8. Model Train__
The model is fitted with the entire training set.

In [ ]:
start_time = time.time()
clf_tree.fit(X_train, y_train)
end_time = time.time()
tree_time = end_time - start_time

start_time = time.time()
clf_knn.fit(X_train, y_train)
end_time = time.time()
knn_time = end_time - start_time

### __9. Evaluation with Test (Outer Evaluation)__
Predictions are made on the test set and the main metrics are calculated:

- Balanced Accuracy: Average of TPR and TNR.
- Accuracy: Proportion of correct predictions.
- TPR (Sensitivity/Recall): How well the model detects positive cases.
- TNR (Specificity): How well the model detects negative cases.
- Confusion Matrix: Detailed visualization of hits and errors.

In [ ]:
d_tree_ev = evaluate_model(clf_tree, X_test, y_test, "Decision Tree (Base)")
knn_ev = evaluate_model(clf_knn, X_test, y_test, "KNN (Base)")

### __10. Model Optimization with RandomizedSearchCV__

Randomized hyperparameter search (_RandomizedSearchCV_) is used to find the best configuration for the models to improve their performance by efficiently tuning their hyperparameters.

- KNN (_n_neighbors_, _weights_, _metric_).  
- Decision Tree (_max_depth_, _criterion_).


In [ ]:
param_dist_knn = {"n_neighbors": randint(1, 20), "weights": ["uniform", "distance"], "metric": ["euclidean", "manhattan"]}
param_dist_tree = {"max_depth": randint(3, 20), "criterion": ["gini", "entropy"]}

random_knn = RandomizedSearchCV(KNeighborsClassifier(), 
                                param_distributions=param_dist_knn, 
                                n_iter=10, cv=5, 
                                scoring="balanced_accuracy", 
                                n_jobs=-1, random_state=100474964)
random_tree = RandomizedSearchCV(DecisionTreeClassifier(), 
                                 param_distributions=param_dist_tree, 
                                 n_iter=10, cv=5, 
                                 scoring="balanced_accuracy", 
                                 n_jobs=-1, random_state=100474964)

start_time = time.time()
random_knn.fit(X_train_transformed, y_train)
end_time = time.time()
knn_hpo_time = end_time - start_time

start_time = time.time()
random_tree.fit(X_train_transformed, y_train)
end_time = time.time()
tree_hpo_time = end_time - start_time

print(f"Best hyperparameters for KNN: {random_knn.best_params_}")
print(f"Balanced Accuracy KNN: {random_knn.best_score_:.4f}")
print(f"Best hyperparameters for Decision Tree: {random_tree.best_params_}")
print(f"Balanced Accuracy Decision Tree: {random_tree.best_score_:.4f}")


##### Tests of Distances and Their Impact on the Model
Various values for distance metrics (_euclidean_ and _manhattan_) have been tested in the KNN model, along with different hyperparameter configurations (_n_neighbors_ and _weights_), to find the optimal values. 

Now, the impact on the performance of the optimized model will be evaluated.


In [ ]:
# Extract results from RandomizedSearchCV for Decision Tree
tree_results = pd.DataFrame(random_tree.cv_results_)
plt.figure(figsize=(10, 5))
sns.lineplot(x=tree_results["param_max_depth"], y=tree_results["mean_test_score"], marker="o", label="Balanced Accuracy")
plt.xlabel("Max Depth (Decision Tree)")
plt.ylabel("Balanced Accuracy")
plt.title("Impact of max_depth on Balanced Accuracy (Decision Tree)")
plt.grid()
plt.show()

# Extract results from RandomizedSearchCV for KNN
knn_results = pd.DataFrame(random_knn.cv_results_)
plt.figure(figsize=(10, 5))
sns.lineplot(x=knn_results["param_n_neighbors"], y=knn_results["mean_test_score"], marker="o", label="Balanced Accuracy")
plt.xlabel("Number of Neighbors (KNN)")
plt.ylabel("Balanced Accuracy")
plt.title("Impact of n_neighbors on Balanced Accuracy (KNN)")
plt.grid()
plt.show()


### __11. Evaluation of Optimized Models__

To confirm that hyperparameter optimization actually improves performance:
- The best hyperparameters are selected and the final models are trained.
- All metrics are recalculated to see the improvement over the default models.


In [ ]:
opt_tree_ev = evaluate_model(random_tree.best_estimator_, X_test_transformed, y_test, "Decision Tree (Optimized)")
opt_knn_ev = evaluate_model(random_knn.best_estimator_, X_test_transformed, y_test, "KNN (Optimized)")


##### 11.1 Evaluate base vs HPO times and metrics

In [ ]:
# Comparative table for evaluation metrics
results = pd.DataFrame([dummy_ev, d_tree_ev, knn_ev, opt_tree_ev, opt_knn_ev],
                       index=["Dummy", "Dec. Tree (Base)", "KNN (Base)", "Dec. Tree (Opt)", "KNN (Opt)"])
print(results, '\n')


# Comparative table for training times
time_comparison = pd.DataFrame({
    "Base": [tree_time, knn_time],
    "HPO": [tree_hpo_time, knn_hpo_time]
}, index=["Decision Tree", "KNN"])

print(time_comparison)

# Comparative bar plot
time_comparison.plot(kind="bar", figsize=(5, 3), legend=True)
plt.ylabel("Training Time (s)", fontsize=10)
plt.title("Comparison of Training Time Between Base and Optimized Models", fontsize=12)
plt.xticks(rotation=0, fontsize=9)
plt.yticks(fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

##### __Conclusions of the Evaluation Comparison__

__Impact of Models on Accuracy and Balance__

(These results have been obtained from a specific evaluation and may vary slightly.)

- The **Dummy Model** has a **Balanced Accuracy of 0.50**, indicating that it is not learning anything and simply predicts the majority class.
- The **Decision Tree (Base)** significantly outperforms the Dummy model, with a **Balanced Accuracy of 0.7998** and a **TPR of 0.6519**, showing that it is better at detecting the minority class.
- The **KNN (Base)** performs worse than the decision tree, with a **Balanced Accuracy of 0.6187** and a **TPR of 0.2848**, indicating difficulties in detecting the minority class.

__Impact of Hyperparameter Optimization (HPO)__
- **Optimized Decision Tree** improves over the base model in **all metrics**, achieving a **Balanced Accuracy of 0.8456** (+4.6%) and a **TPR of 0.7215** (+6.9%), indicating better detection of the minority class without losing precision in the majority class.
- **Optimized KNN** improves significantly, with a **Balanced Accuracy of 0.8408** (+22.2%) and a **TPR of 0.6962** (+41.1%), showing that optimization has made KNN much more effective at detecting the minority class.

__Impact on Training Time__
- The **Base Decision Tree** trains in **0.1218 s**, while its optimized version takes **0.5518 s** (**4.5 times slower**), but with a significant improvement in performance.
- The **Base KNN** trains in **0.0472 s**, while the optimized version increases to **0.4834 s** (**10.2 times slower**), although the improvement in TPR and Balanced Accuracy justifies this increase in time.

##### __Final Conclusion__
1. Decision Tree is the best overall model, with a Balanced Accuracy of 0.8456 after optimization and good detection of the minority class.
2. KNN improves significantly with HPO, achieving a competitive TPR, although it remains more computationally expensive.
3. Training time for optimized models increases, but it remains manageable for Decision Tree, while for KNN it is considerably higher.
4. HPO optimization is effective and justified, as it improves minority class detection and overall accuracy without an excessive computational cost.


## __Use of AI in the Development of the Work__
In the development of this work, artificial intelligence (ChatGPT) has been used as a support tool to facilitate the understanding of concepts, receive methodological and code suggestions, and improve the documentation writing.

#### __1. Generation of Ideas and Recommendations__

For project planning and model implementation, AI has been used to obtain methodological suggestions and alternative approaches in the following aspects:

- Selection of preprocessing methods: Different strategies for data imputation, variable scaling, and categorical data encoding have been explored.
- Model comparison: Recommendations have been received on how to structure the evaluation of base models versus optimized models through hyperparameter tuning.
- Code improvement: It has helped improve and refactor some code blocks to enhance their efficiency and readability.

#### __2. Explanation of Concepts__

At certain points during the development, AI has been consulted for detailed explanations on technical aspects such as:

- Meaning and interpretation of metrics like balanced accuracy, TPR, and TNR.
- Differences between internal evaluation (inner evaluation) and external evaluation (outer evaluation).
- Hyperparameter optimization using RandomizedSearchCV and its application in model selection.

#### __3. Support in Documentation Writing__
For the preparation of the report, AI has served as an assistant in:

- Organizing sections and document structure.
- Writing clearer and more concise explanations.
- Formulating conclusions based on the obtained results.
- Style correction and clarity in the exposition of ideas.
- Writing the section 'Use of AI in the Development of the Work'

The use of AI in this work has been strictly as support in the learning process, idea generation, and documentation writing, always supervising and verifying the results without replacing the execution of analyses, code implementation, or result interpretation. All final decisions have been made autonomously, based on the analysis and understanding of the problem.
